In [1]:
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import euclidean_distances
from math import sqrt
import matplotlib.pyplot as plt
import networkx as nx
import random
import gudhi

In [ ]:
class geometric_network():
    def __init__(self, network_type, size, **kwargs):
        ####################################
        ##### Geometric Network Object #####
        ####################################
        
        ### input -- network_type:
        ##                      2D_lattice: creates 2d lattice of given tiling-can be triangular, square, hexagonal
        #                                   --Additional Arguments--
        #                                     tiling = 3,4,6
        #                                     periodic = bool (connects the edge of the grid to the opposite edge)
        ##                      ring_lattice: creates a ring graph with each node degree 2
        #                                    --Additional Arguments--
        #                                     banded = bool
        #                                     band_length = gonna connect this many closest neigbors (skipping the 
        #                                                   adjacent nodes from both sides because they are already 
        #                                                   connected) from both sides to every node
        #                                     
        #                                     
        ### input -- size: network size, if 2D_lattice, then there will be size^2 many nodes.
        
        ### attribute -- geometric_network.N
        ### attribute -- geometric_network.M
        ### attribute -- geometric_network.graph
        ### attribute -- geometric_network.pos
        ### attribute -- geometric_network.A
        ### attribute -- geometric_network.text 
        
        ### function -- geometric_network.display(n_size, labels = Bool): take a sneakpeak on the networkx object
        ### function -- geometric_network.add_noise_to_geometric(noise_type, d2): add two types of noise
        #                                   either 'ER_like' or 'k_regular' of degree d2
        ### function -- geometric_network.run_excitation(Trials, p, T, seed, threshold): run 
        #                                   the excitation according to these parameters
        
        ### function -- geometric_network.spy_first_activation(first_activation_times): visualize the first
        #                                   activation, input of the functin is the 2nd output of run_excitation
        
        if network_type == '2D_lattice': 
            tiling = kwargs['tiling']
            per = kwargs['periodic']
            if tiling == 3: 
                self.graph = nx.triangular_lattice_graph(size, size, periodic = per, with_positions = True)
                self.pos = nx.get_node_attributes(self.graph,'pos')
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
                

            elif tiling == 4: 
                self.graph = nx.grid_2d_graph(size, size, periodic = per)
                self.pos = dict( (n, n) for n in self.graph.nodes() )
                self.labels = dict( ((i, j), i * size + j) for i, j in self.graph.nodes() )
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
                
            elif tiling == 6: 
                self.graph = nx.hexagonal_lattice_graph(size, size, periodic = per, with_positions = True)
                self.pos = nx.get_node_attributes(self.graph,'pos')
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
            
            
        elif network_type == 'ring_lattice':# TODO: banding for every node
            self.graph = nx.cycle_graph(size)
            theta = (2*np.pi)/size
            self.pos = dict((i,(np.sin(theta*i),np.cos(theta*i))) for i in range(size))
            self.M = len(self.graph.edges())
            self.N = len(self.graph.nodes())
            self.text = 'Ring Lattice'
            if kwargs['banded']:
                if kwargs['band_length'] >= int(self.N/2)-1: raise ValueError('Band length cannot exceed the half of the size of the network')
                if kwargs['band_length'] <2: raise ValueError('Band length should be a positive integer greater 1 since the closest neighbors are already connected')
                for u in range(self.N):
                    for i in range(2,kwargs['band_length']+1):# ranges from 2 to k+2 to avoid the closest node and start
                        ## banding from the second closest node
                        if u + i >= self.N: v = u + i - self.N
                        else: v = u + i
                        self.graph.add_edge(u, v)
                        if u - i < 0: v = self.N + u - i
                        else: v = u - i
                        self.graph.add_edge(u, v)
                self.text = self.text + ' w/ bandlength %d'%kwargs['band_length']
            else:self.text = self.text + ' w/ bandlength 0'
                        
        else: raise ValueError('network type can be a lattice or a ring')
            
        self.A = nx.adjacency_matrix(self.graph)## use .toarray() or .todense() attributes to call the sparse matrix
              
    def display(self, n_size = 15, labels = True):
        
        try: return(nx.draw(self.graph, pos = self.pos, node_size = n_size, with_labels = labels, font_weight = 'bold'))
        
        except: return(nx.draw(self.graph, with_labels = labels, font_weight = 'bold'))
        
    def add_noise_to_geometric(self, noise_type, d2):
        
        # this function adds nongeometric edges to the network that are long range. Depending on the 'noise_type'
        # the way we add these long range edges differ. If noise_type = ER_like, then there will be d2 many
        # non geometric edges ON AVARAGE for every node. When the noise_type = k_regular, every node will 
        # have exactly d2 many long range edges
        
        
        if noise_type == 'ER_like':   
            
            M = int(self.N * d2)
        
            if M%2 == 1: raise ValueError('Try providing an even non-geometric degree')
                
            edges_built = 0
            
            while edges_built <= M/2:
                
                nodes_to_connect = [random.randint(0,self.N-1) for i in range(2)]
                
                if nodes_to_connect[0] == nodes_to_connect[1]: #no self-loops
                    edges_built = edges_built
                elif self.A.toarray()[nodes_to_connect[0]][nodes_to_connect[1]] == 1 or self.A.toarray()[nodes_to_connect[1]][nodes_to_connect[0]] == 1: # no existing loops
                    edges_built = edges_built
                else: 
                    self.graph.add_edge(nodes_to_connect[0],nodes_to_connect[1])
                    edges_built = edges_built + 1
            self.text = self.text + 'and ER-like noise w/ degree %d'%d2

                    
        elif noise_type == 'k_regular':
            
            self.text = self.text + 'and k-regular noise w/ degree %d'%d2
            
            M = int(self.N * d2)
        
            if M%2 == 1: raise ValueError('Try providing an even non-geometric degree')
            
            flag_2 = True
            
            while flag_2:
                flag_2 = False
                #build stubs
                stubs = np.zeros(M)
                for i in range(self.N):
                    index = (i*d2) + np.arange(d2)
                    stubs[index[0]:index[-1]+1] = (i) * np.ones(d2)
                    
                #build undirected link list
                link_list = np.zeros((int(M/2),2))
                for m in range(int(M/2)):
                    flag_1 = True # turn on flag to enter while loop
                    count = 0
                    while flag_1:
                        flag_1 = False #turn off flag to exit while loop
                        rand = [random.randint(0,len(stubs)-1) for i in range(2)]
                        
                        node_A = int(stubs[rand[0]])
                        node_B = int(stubs[rand[1]])
                                                
                        if node_A == node_B: flag_1 = True
                        
                        for n in range(m):
                            if link_list[n,0] == node_A and link_list[n,1] == node_B:
                                flag_1 = True
                            if link_list[n,0] == node_B and link_list[n,1] == node_A:
                                flag_1 = True
                            if self.A.toarray()[node_A][node_B] == 1 or self.A.toarray()[node_B][node_A] == 1:
                                flag_1 = True
                                
                        count = count +1
                        
                        if count > M: flag_2 = True ; break
                            
                    #make link       
                    link_list[m,0] = node_A
                    link_list[m,1] = node_B
                    
                    #remove stubs from list
                    stubs = np.delete(stubs,[rand[0],rand[1]])
            
            #build network
            for m in range(int(M/2)):
                self.graph.add_edge(link_list[m,0],link_list[m,1])
            
        else: raise InputError('Noise type can only be ER_like or k_regular')
            
        self.A = nx.adjacency_matrix(self.graph)
    
    def excitation(self, T, C, seed, threshold, refractory = False, ax = None, spy = False):
        
                            ### THE CORE FUNCTION OF THE NEURONAL CONTAGION MODEL###
        
        # In this model, a neuron fires if the ratio of it's excited neighbors to the total number of neighbors
        # is greater than the threshold. Let's call the difference between this ratio and the threshold = F 
        # so that if F is positive, neuron is going to fire and it doesn't fire when it's negative. We add some 
        # stocasticity to the model by defining the sigmoid function so that the probability that the 
        # neuron is going to fire is not a step function, but a sigmoid function.
        
        ################################################################################################
        
        #### Inputs : 
        ##           T: Number of time steps contagions is going to be iterated
        
        ##           C: A positive constant for the sigmoid function, if C is too large(>100), 
        #               jump from 0 to 1 is gonna be too quick i.e. model is going to be deterministic
        
        ##          seed: node id to start the contagion, in the first time step, we infect 
        #                 the neighbors of the seed with probablity 1 then enter the while loop below
        
        ##          threshold: threshold to compare for a neuron's neighbor input
        
        ##          refractory: if TRUE, sets the refractory period of 1 time step i.e. neuron cannot fire for 
        #                      1 time step right after it fires. if FALSE, neuron stays active once its activated.
        
        ##          ax, spy: if spy is TRUE, there have to be an axis provided to plot the contagion spread.
        
        #### Outputs : 
        ##          activation_times: An array of n x 1 keeping track of the first time step the corresponding 
        #                             node gets activated.
        
        ##          size_of_contagion: An array of (T+1) x 1 keeping track of the number of active nodes at a
        #                              given time(at t = 0, all neighbors of the seed is active)
        
        #################################################################################################
        
        n = self.N
        x_1 = sparse.lil_matrix(((n,T+1)), dtype = bool)
        activation_times = np.ones(n, dtype = int)*T
        
        excited_nodes = self.A[seed].nonzero()[1]
        
        deg = len(excited_nodes)
        
        size_of_contagion = np.zeros(T+1)
        
        vals = [True] * deg
        
        t = 0
        
        for i in range(deg): 
            x_1[excited_nodes[i],t] = vals[i]
            
        all_nodes = set([k for k in range(n)])
        size_of_contagion[t] = len(excited_nodes)
        
        while t<T and len(excited_nodes)>0: # and np.any(activation_times==T):
            x_old = x_1.toarray()[:,t]
            excited_nodes = np.nonzero(x_old)[0]         
            activation_times[excited_nodes] = np.minimum(activation_times[excited_nodes], 
                                                         np.array([t]*len(excited_nodes)))
            activation_times = activation_times.astype(np.int32)
            t+=1
            for j in list(all_nodes-set(excited_nodes)): 
                ids = np.nonzero(self.A[j])[1]
                F = np.sum(x_old[ids])/len(ids) - threshold   
                Z = 1/(1+np.exp(-C*F)) 
                rand = random.uniform(0,1)
                x_1[j,t] = bool(rand<Z) 
                
            if not refractory: x_1[x_old,t] = True 
            
            size_of_contagion[t] = np.sum(x_1[:,t])
      
        if spy: ax.imshow(x_1.todense())
        return(activation_times, size_of_contagion)
    
    
    def run_excitation(self, Trials, T, C, seed, threshold, refractory, ax = None):
        
        n = self.N
        
        first_excitation_times = np.zeros((n,Trials))
        size_of_contagion = np.zeros((T+1,Trials))
        
        if ax is not None: 
            for i in range(Trials):
    
                first_exct, contagion_size = self.excitation(T = T, 
                                                             C = C, 
                                                             seed = seed, 
                                                             threshold = threshold, 
                                                             refractory = refractory,
                                                             ax = ax[i], 
                                                             spy = True)
                first_excitation_times[:,i] = first_exct
                size_of_contagion[:,i] = contagion_size
                ax[i].set_title('Trial %d for %s'%(i+1, self.text), fontsize = 25)
                ax[i].set_xlabel('Time', fontsize = 35)
                ax[i].set_ylabel('Node ID',fontsize = 35)
                ax[i].set_xticks([5*t for t in range(int(T/5)+1)])
                ax[i].set_yticks([10*t for t in range(int(n/10))])
                ax[i].tick_params('both', which = 'major', labelsize = 15)
            plt.tight_layout()
            
        else:
            for i in range(Trials):
    
                first_exct, contagion_size = self.excitation(T = T, 
                                                             C = C, 
                                                             seed = seed, 
                                                             threshold = threshold, 
                                                             refractory = refractory)
                first_excitation_times[:,i] = first_exct
                size_of_contagion[:,i] = contagion_size
            
        return(first_excitation_times, size_of_contagion)
    

    def spy_first_activation(self, first_activation_times):
        
        fig,ax = plt.subplots(1,1,figsize = (10,8))
        
        pos = ax.imshow(first_activation_times, 
                        interpolation = 'nearest', 
                        aspect = 'auto', 
                        cmap = 'viridis', 
                        extent = [0.5,Trials+0.5,0,n])
        ax.set_title('First Activation Times for %s'%self.text, fontsize = 20)
        ax.set_xlabel('Trials', fontsize = 15)
        ax.set_ylabel('Node ID', fontsize = 15)
        ax.set_xticks([i for i in range(1,Trials+1)])
        
        fig.colorbar(pos, ax = ax)
        
    def average_over_trials(self, matrix): ##matrix to take the average over trials
        ## matrix have to be k x T where T is the number of trials
        return(np.mean(matrix, axis = 1))
    
    def make_distance_matrix(self, T, C, threshold, Trials, refractory, spy_distance = False):
        n = self.N
        D = np.zeros((n,n))
        Q = np.zeros((n,T+1))
        
        for i in range(n):
            fat, qs = self.run_excitation(Trials, T, C, i, threshold, refractory)
            D[i], Q[i] = self.average_over_trials(fat), self.average_over_trials(qs)
        
        D1 = euclidean_distances(D.T)
        
        if spy_distance: 
            fig,ax = plt.subplots(1,1, figsize = (15*int(n/20),10*int(n/20)))
            pos = ax.imshow(D1, 
                            origin = 'lower', 
                            interpolation = 'nearest', 
                            aspect = 'auto', 
                            cmap = 'viridis', 
                            extent = [-0.5,n-0.5,-0.5,n-0.5])
            ax.set_title('Distance Matrix for %s'%self.text, fontsize = 20)
            ax.set_ylabel('Excitation starting from node i', fontsize = 15)
            ax.set_xlabel('First time node j gets activated', fontsize = 15)
            ax.set_xticks([i*5 for i in range(int(n/5))])
            ax.set_yticks([i*5 for i in range(int(n/5))])

            fig.colorbar(pos, ax = ax)
                        
        return(D1, Q)
    
    def compute_persistence(self, distances, spy = False):    
        rips_complex = gudhi.RipsComplex(distance_matrix = distances/np.max(distances), max_edge_length = 1)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension = 2)
        diag = simplex_tree.persistence(min_persistence = 0.0)
        if spy: 
            gudhi.plot_persistence_diagram(diag)
        return(diag)
    
    def one_d_Delta(self, persistence):
        oned_holes = [(0,0)]
        for i in range(len(persistence)):
            if persistence[i][0] == 1:
                oned_holes.append(persistence[i][1])
        oned_holes = np.array(oned_holes)
        persistence_life_times = oned_holes[:,1]-oned_holes[:,0]
        Delta_min = np.sort(persistence_life_times)[-1]-np.sort(persistence_life_times)[-2]
        Delta_max = np.sort(persistence_life_times)[-1]-np.sort(persistence_life_times)[1]
        Delta_avg = np.mean(persistence_life_times[1:])
        return(Delta_min, Delta_max, Delta_avg)

In [ ]:
def display_comm_sizes(Q, labels):
    argmaxs = []
    
    for i in range(len(Q)):
        Q_mean = np.mean(Q[i], axis = 0)
        argmaxs.append(np.argmax(Q_mean))
    
    fig,ax = plt.subplots(1,1, figsize =(20,10))
    
    for i in range(len(Q)):  
        Q_mean = np.mean(Q[i], axis = 0)
        ax.plot(Q_mean[:int(np.max(argmaxs)+10)], 
                label = '%s'%labels[i], marker = 'v')
        
        X = np.linspace(0, int(np.max(argmaxs)+9), int(np.max(argmaxs)+10))
        ax.fill_between(X, 
                        np.max(Q[i], axis = 0)[:int(np.max(argmaxs)+10)], 
                        np.min(Q[i], axis = 0)[:int(np.max(argmaxs)+10)], 
                        alpha = 0.2)
        
    ax.set_title('Size of the Active Nodes', fontsize = 25)
    ax.set_xlabel('Time', fontsize = 20)
    ax.set_ylabel('Number of Nodes', fontsize = 20)
    ax.legend(fontsize = 'x-large')

In [4]:
n = 200
refractory_period = False
T = 100 # number of iterations
thresholds = np.linspace(0,0.6,31) # resistence of the node to it's neighbors' excitation level
Trials = 1 # number of trials
d2s = [1,2,3,4,5,6,7,8,9,10]
bands = 5
C = 10000

delta_matrix_min = np.zeros((len(thresholds),len([0]+d2s)))
delta_matrix_max = np.zeros((len(thresholds),len([0]+d2s)))
delta_matrix_avg = np.zeros((len(thresholds),len([0]+d2s)))

all_deltas_min = {}
all_deltas_max = {}
all_deltas_avg = {}

In [5]:
thresholds

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 ])

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.00/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[0], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[0][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[0][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[0][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[0]] = delta_mins
all_deltas_max['%.3f'%thresholds[0]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[0]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[0]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.02/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[1], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[1][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[1][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[1][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[1]] = delta_mins
all_deltas_max['%.3f'%thresholds[1]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[1]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[1]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.04/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[2], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[2][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[2][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[2][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[2]] = delta_mins
all_deltas_max['%.3f'%thresholds[2]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[2]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[2]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.06/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[3], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[3][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[3][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[3][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[3]] = delta_mins
all_deltas_max['%.3f'%thresholds[3]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[3]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[3]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.08/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[4], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[4][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[4][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[4][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[4]] = delta_mins
all_deltas_max['%.3f'%thresholds[4]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[4]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[4]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.10/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[5], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[5][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[5][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[5][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[5]] = delta_mins
all_deltas_max['%.3f'%thresholds[5]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[5]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[5]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.12/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[6], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[6][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[6][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[6][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[6]] = delta_mins
all_deltas_max['%.3f'%thresholds[6]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[6]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[6]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.14/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[7], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')

    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[7][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[7][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[7][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[7]] = delta_mins
all_deltas_max['%.3f'%thresholds[7]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[7]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker = 'o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average LifeTime', marker = 'o'  )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[7]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.20/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)

    delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[3], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[3][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[3][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[3][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[3]] = delta_mins
all_deltas_max['%.3f'%thresholds[3]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[3]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[3]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.25/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[4], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[4][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[4][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[4][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[4]] = delta_mins
all_deltas_max['%.3f'%thresholds[4]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[4]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[4]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.30/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[5], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[5][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[5][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[5][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[5]] = delta_mins
all_deltas_max['%.3f'%thresholds[5]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[5]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[5]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.35/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[6], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[6][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[6][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[6][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[6]] = delta_mins
all_deltas_max['%.3f'%thresholds[6]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[6]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[6]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.40/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[7], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[7][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[7][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[7][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[7]] = delta_mins
all_deltas_max['%.3f'%thresholds[7]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[7]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[7]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.45/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[8], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[8][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[8][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[8][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[8]] = delta_mins
all_deltas_max['%.3f'%thresholds[8]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[8]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[8]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.50/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[9], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[9][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[9][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[9][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[9]] = delta_mins
all_deltas_max['%.3f'%thresholds[9]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[9]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[9]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/thresh=0.55/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
delta_mins = []
delta_maxs = []
delta_avgs = []
distance_matrices = []
comm_sizes = []

for i, d2 in enumerate([0]+d2s):
    D,Q = networks[i].make_distance_matrix(T, C, thresholds[10], Trials, refractory_period, spy_distance = True)
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    
    distance_matrices.append(D)
    comm_sizes.append(Q)
    
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    
    delta_min, delta_max, delta_avg = networks[i].one_d_Delta(pers)
    
    delta_mins.append(delta_min)
    delta_matrix_min[10][i] = delta_min
    
    delta_maxs.append(delta_max)
    delta_matrix_max[10][i] = delta_max
    
    delta_avgs.append(delta_avg)
    delta_matrix_avg[10][i] = delta_avg

display_comm_sizes(comm_sizes,[0]+d2s)
plt.savefig(path + 'contagion_size.pdf')
    
all_deltas_min['%.3f'%thresholds[10]] = delta_mins
all_deltas_max['%.3f'%thresholds[10]] = delta_maxs
all_deltas_avg['%.3f'%thresholds[10]] = delta_avgs

fig,ax = plt.subplots(1, 1, figsize = (10,10))  

ax.plot(np.array([0]+d2s)/(2*bands), delta_mins, label = 'Delta Lifetime of longest two cycles', marker = 'o')
ax.plot(np.array([0]+d2s)/(2*bands), delta_maxs, label = 'Delta Lifetime of longest and shortest cycle', marker ='o' )
ax.plot(np.array([0]+d2s)/(2*bands), delta_avgs, label = 'Average Lifetime', marker = 'o' )

fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, Threshold = %.3f'%(n, 2*bands, T , Trials, C, thresholds[10]))
ax.set_title('Deltas vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Deltas')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize = (8,8))
for i,e in enumerate(thresholds):
    ax.plot(np.array([0]+d2s)/(2*bands), all_deltas_min['%.3f'%e], label = 'Threshold = %.3f'%e, marker = 'X' )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, refractory period = %d'%(n, 2*bands, T , Trials, C, int(refractory_period)))
ax.set_title('Delta Min vs nGD/GD for varying Thresholds')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta Minimum')
ax.legend(fontsize = 'large')
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/all_delta_mins.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize = (8,8))
for i,e in enumerate(thresholds):
    ax.plot(np.array([0]+d2s)/(2*bands), all_deltas_max['%.3f'%e], label = 'Threshold = %.3f'%e, marker = 'X' )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, refractory period = %d'%(n, 2*bands, T , Trials, C, int(refractory_period)))
ax.set_title('Delta Max vs nGD/GD for varying Thresholds')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta Maximum')
ax.legend(fontsize = 'large')
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/all_delta_maxs.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize = (8,8))
for i,e in enumerate(thresholds):
    ax.plot(np.array([0]+d2s)/(2*bands), all_deltas_avg['%.3f'%e], label = 'Threshold = %.3f'%e, marker = 'X' )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, C = %d, refractory period = %d'%(n, 2*bands, T , Trials, C, int(refractory_period)))
ax.set_title('Delta Avg vs nGD/GD for varying Thresholds')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta Average')
ax.legend(fontsize = 'large')
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/all_delta_avgs.pdf')

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (12,12))
im = ax.imshow(delta_matrix_min, 
               origin = 'lower', 
               cmap = 'gist_heat', 
               extent = [-.05, (d2s[-1])/(2*bands)+0.05, thresholds[0]-0.005, thresholds[-1]+0.005])
fig.suptitle('N=%d, GD = %d,  T = %d, Trials = %d, C = %d'%(n,2*bands,T, Trials, C),fontsize = 25)
ax.set_title('Delta Mins vs nGD/GD', fontsize = 20)
ax.set_xticks(np.array([0]+d2s)/(2*bands))
ax.set_yticks(thresholds)
ax.set_xlabel('$alpha$ = nGD/GD', fontsize = 15 )
ax.set_ylabel('Thresholds', fontsize = 15)
ax.tick_params(axis = 'both', labelsize = 12)
fig.colorbar(im)
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/delta_matrix_mins.pdf')

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (12,12))
im = ax.imshow(delta_matrix_max, 
               origin = 'lower', 
               cmap = 'gist_heat', 
               extent = [-.05, (d2s[-1])/(2*bands)+0.05, thresholds[0]-0.005, thresholds[-1]+0.005])
fig.suptitle('N=%d, GD = %d,  T = %d, Trials = %d, C = %d'%(n,2*bands,T, Trials, C),fontsize = 25)
ax.set_title('Delta Maxs vs nGD/GD', fontsize = 20)
ax.set_xticks(np.array([0]+d2s)/(2*bands))
ax.set_yticks(thresholds)
ax.set_xlabel('$alpha$ = nGD/GD', fontsize = 15 )
ax.set_ylabel('Thresholds', fontsize = 15)
ax.tick_params(axis = 'both', labelsize = 12)
fig.colorbar(im)
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/delta_matrix_maxs.pdf')

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (12,12))
im = ax.imshow(delta_matrix_avg, 
               origin = 'lower', 
               cmap = 'gist_heat', 
               extent = [-.05, (d2s[-1])/(2*bands)+0.05, thresholds[0]-0.005, thresholds[-1]+0.005])
fig.suptitle('N=%d, GD = %d,  T = %d, Trials = %d, C = %d'%(n,2*bands,T, Trials, C),fontsize = 25)
ax.set_title('Delta Avgs vs nGD/GD', fontsize = 20)
ax.set_xticks(np.array([0]+d2s)/(2*bands))
ax.set_yticks(thresholds)
ax.set_xlabel('$alpha$ = nGD/GD', fontsize = 15 )
ax.set_ylabel('Thresholds', fontsize = 15)
ax.tick_params(axis = 'both', labelsize = 12)
fig.colorbar(im)
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/excitations_WTM_no_refractory/delta_matrix_avgs.pdf')